In [1]:
pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 5.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 7.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 7.9 MB/s eta 0:00:00
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.5 MB/s eta 0:00:00
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.8/823.8 kB 6.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 7.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 6.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('model_50_epochs.pt') 

# Access the webcam
cap = cv2.VideoCapture(0)  

# Define a dictionary to map class IDs to card labels
class_labels = {
    0: ('10C', 10),
    1: ('10D', 10),
    2: ('10H', 10),
    3: ('10S', 10),
    4: ('2C', 2), 
    5: ('2D', 2),
    6: ('2H', 2), 
    7: ('2S', 2),
    8: ('3C', 3),
    9: ('3D', 3),
    10: ('3H', 3),
    11: ('3S', 3),
    12: ('4C', 4),
    13: ('4D', 4),
    14: ('4H', 4),
    15: ('4S', 4),
    16: ('5C', 5),
    17: ('5D', 5),
    18: ('5H', 5),
    19: ('5S', 5),
    20: ('6C', 6),
    21: ('6D', 6),
    22: ('6H', 6),
    23: ('6S', 6),
    24: ('7C', 7),
    25: ('7D', 7),
    26: ('7H', 7),
    27: ('7S', 7),
    28: ('8C', 8),
    29: ('8D', 8),
    30: ('8H', 8),
    31: ('8S', 8),
    32: ('9C', 9),
    33: ('9D', 9),
    34: ('9H', 9),
    35: ('9S', 9),
    36: ('AC', 11), # Ace can also be considered 1 and it needs to be coded accordingly
    37: ('AD', 11),
    38: ('AH', 11),
    39: ('AS', 11),
    40: ('JC', 10),
    41: ('JD', 10),
    42: ('JH', 10),
    43: ('JS', 10),
    44: ('KC', 10),
    45: ('KD', 10),
    46: ('KH', 10),
    47: ('KS', 10),
    48: ('QC', 10),
    49: ('QD', 10),
    50: ('QH', 10),
    51: ('QS', 10)
}

# Initialize dictionaries to store detected cards and their frame counts
detected_cards_count = {}
detected_cards = set()
sum_detected_cards = 0
consecutive_frame_threshold = 10

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Inference
    results = model(frame)

    # Get the annotated frame with detections
    annotated_frame = results[0].plot()
    
    # Temporarily store the current frame's detected cards
    current_frame_cards = set()

    # Loop through the detections
    for box in results[0].boxes:
        class_id = int(box.cls[0])
        label = class_labels.get(class_id, 'Unknown')
        current_frame_cards.add(label)
    
    # Update the detected cards count
    for card in current_frame_cards:
        if card in detected_cards_count:
            detected_cards_count[card] += 1
        else:
            detected_cards_count[card] = 1
    
    # Remove cards that were not detected in the current frame
    cards_to_remove = [card for card in detected_cards_count if card not in current_frame_cards]
    for card in cards_to_remove:
        detected_cards_count[card] -= 1
        if detected_cards_count[card] <= 0:
            detected_cards_count.pop(card)

    # Add cards to the detected cards set if they appear for 3 consecutive frames
    for card, count in detected_cards_count.items():
        if count >= consecutive_frame_threshold:
            detected_cards.add(card)
        else:
            detected_cards.discard(card)

    # Display the detected cards at the top right corner
    y_offset = 20
    for card_label in detected_cards:
        cv2.putText(annotated_frame, f'Card: {card_label[0]}', (annotated_frame.shape[1] - 300, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        y_offset += 20
    
    # Summing the displayed cards
    sum_detected_cards = sum(value for _, value in detected_cards)
    cv2.putText(annotated_frame, f'Total Value: {sum_detected_cards}', (annotated_frame.shape[1] - 300, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Displaying the message about BlackJack or Explosion
    if sum_detected_cards > 21:
            message = 'Exploded!'
    elif sum_detected_cards == 21:
            message = 'BLACKJACK!'
    else:
            message = ''

    if message:
        # Get the size of the message text
        screen_height = annotated_frame.shape[0]
        screen_width = annotated_frame.shape[1]
        font_scale = screen_height / 5 / 20  # Adjust font scale based on 20% of screen height
        text_size = cv2.getTextSize(message, cv2.FONT_HERSHEY_DUPLEX, font_scale, 2)[0]

        # Calculate the center position for the text
        text_x = (screen_width - text_size[0]) // 2
        text_y = (screen_height + text_size[1]) // 2

        # Draw the message in the middle of the screen
        cv2.putText(annotated_frame, message, (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX, font_scale, (0, 0, 255) if message == 'Exploded!' else (0, 255, 255), 2)

    # Display the resulting frame with annotations
    cv2.imshow('YOLOv8 Webcam', annotated_frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


c:\Users\Julio\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0: 480x640 (no detections), 82.0ms
Speed: 2.0ms preprocess, 82.0ms inference, 12.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 BACK, 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 34.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 BACK, 4.0ms
Speed: 2.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 BACK, 4.5ms
Speed: 1.0ms preprocess, 4.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at

ValueError: too many values to unpack (expected 2)

: 